# <span style="color:red">Problemi di localizzazione</span>


## <span style="color:blue">Definizione:</span>
### I modelli di localizzazione sono uno dei principali strumenti quantitativi per la pianificazione territoriale di reti di servizio.

## <span style="color:blue">Obiettivo:</span>
### Definire le localizzazioni di centri di servizio per soddisfare una domanda dispersa sul territorio.

## <span style="color:blue">Osservazioni:</span>
### La scelta dei siti per la localizzazione degli impianti e l'allocazione della domanda ad ogni sito deve avvenire simultaneamente.

## <span style="color:blue">Localizzazione puntuale:</span>
### Si ha se il poblema può essere ricondotto alla ricerca di uno o più punti dove posizionare i centri di servizio.

# <span style="color:red">Problema di P-mediana</span>


## <span style="color:blue">Definizione:</span>
### Nel problema di P-mediana i costi di localizzazione sono uguali per ogni centro di servizio. Si stabilisce un numero P di localizzazioni da determinare trascurando, nella funzione obiettivo, i costi di localizzazione.

## <span style="color:blue">Obiettivo:</span>
### Con il problema di P-mediana si ha l'obiettivo di individuare i P nodi nei quali localizzare i centri di servizio minimizzando la somma dei costi di afferenza.

## <span style="color:blue">Formulazione:</span>
### Dati del problema:
### -  <span style="color:purple">$c_{ij}$</span>: costo di afferenza in i della domanda j.

### Variabili di decisione:
### -  <span style="color:purple">$x_{ij}$</span>: se il cliente j afferisce al centro di servizio i, 0 altrimenti.
### -  <span style="color:purple">$y_{i}$</span>: se in i è localizzato un centro di servizio, 0 altrimenti. (<span style="color:purple">$ y_{i} \in \{ 0,1 \}$</span>)

### Funzione obiettivo:

\begin{equation}
\text{Min} \quad Z = \sum_{(i \in I,j \in J)} c_{ij} \cdot x_{ij}
\end{equation}

### Vincolo sul numero di centri da aprire:

\begin{equation}
\sum_{(i \in I)} y_{i} = p
\end{equation}

### Vincolo di assegnamento:

\begin{equation}
\sum_{(i \in I)} x_{ij} = 1 \quad \forall j \in J
\end{equation}

### Vincolo di upper bound variabile:

\begin{equation}
x_{ij} \leq y_{i} \quad \forall i \in I, \forall j \in J
\end{equation}

### Vincolo di interezza (opzionale se dichiarate come binarie):

\begin{equation}
x_{ij} \geq 0 \quad \forall i \in I, \forall j \in J
\end{equation}

# <span style="color:red">Svolgimento:</span>
### Lettura da file dei dati:

In [1]:
import json


def open_file_json(filename):
    with open('./Datasheet/' + filename + '.json', encoding='utf-8') as f:
        cities_json = json.load(f)
    cities_names = []
    cities_coords = {}
    for city in cities_json:
        if not(city['city'] in cities_names):
            cities_names.append(city['city'])
            cities_coords[city['city']] = (float(city['lat']), float(city['lng']))   
    print("Caricati " + str(len(cities_names)) + " nodi")
    return cities_names, cities_coords

# Inserire come argomento della funzione il nome del file .json
cities_names, cities_coords = open_file_json("it_red")

Caricati 1311 nodi


### Generazione della matrice delle distanze (costi di afferenza):

In [2]:
import math


def distance_calculator(c1, c2, cities_coords):
    city_1 = cities_coords[c1]
    city_2 = cities_coords[c2]
    diff = (city_1[0] - city_2[0], city_1[1] - city_2[1])
    return math.sqrt(diff[0] * diff[0] + diff[1] * diff[1])


def distance_matrix_generator(cities_names, cities_coords):
    distance_matrix = {}
    for city_1 in cities_names:
        for city_2 in cities_names:
            if city_1 != city_2:   
                distance_matrix[(city_1, city_2)] = distance_calculator(city_1, city_2, cities_coords)
            else:
                distance_matrix[(city_1, city_2)] = 0.0
    return distance_matrix

distance_matrix = distance_matrix_generator(cities_names, cities_coords)

### Impostazione della variabile P (numeri di centri da localizzare):

In [3]:
p = 6

### Inizializzazione del modello e definizione delle variabili:

In [4]:
import gurobipy as gp
from gurobipy import GRB

Mod = gp.Model("P-Median")

Xvars = Mod.addVars(distance_matrix.keys(), obj=distance_matrix, vtype=GRB.CONTINUOUS, name="x")
Yvars = Mod.addVars(cities_names, vtype=GRB.BINARY, name="y")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-02


### Vincolo sul numero di centri da aprire:

In [5]:
Constr_1 = Mod.addConstr(Yvars.sum() == p)

### Vincolo di assegnamento:

In [6]:
Constr_2 = Mod.addConstrs(Xvars.sum('*',j) == 1 for j in cities_names)

### Vincolo di upper bound variabile:

In [7]:
Constr_3 = Mod.addConstrs(Xvars[(i,j)] <= Yvars[i] for i,j in distance_matrix)

### Vincolo di interezza:

In [8]:
Constr_4 = Mod.addConstrs(Xvars[(i,j)] >= 0 for i,j in distance_matrix)

### Salvataggio del modello in un file lp:

In [9]:
#Mod.write("p_median_model.lp")

### Risoluzione del modello tramite Gurobi:

In [11]:
Mod.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3438754 rows, 1720032 columns and 6876195 nonzeros
Model fingerprint: 0x6b2e20e4
Variable types: 1718721 continuous, 1311 integer (1311 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolved: 1720033 rows, 1720032 columns, 5157474 nonzeros

Continuing optimization...


Explored 1 nodes (265110 simplex iterations) in 0.39 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 2: 1250.72 5347.1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.250720489942e+03, best bound 1.250720489942e+03, gap 0.0000%


### Prelievo e stampa delle soluzioni:


In [12]:
Yvals = Mod.getAttr('x', Yvars)
Xvals = Mod.getAttr('x', Xvars)

medians = []
for i in Yvals.items():
    if i[1] == 1:
        medians.append(i[0])
z = 0
for i, j in Xvals.items():
    if j == 1:
        z = z + distance_matrix[i]

print("Best Solution: " + str(z))
print("Best Medians: ", medians)


Best Solution: 1250.7204899419735
Best Medians:  ['Campi Bisenzio', 'Massafra', 'Valguarnera Caropepe', 'Teano', 'Camposampiero', 'Baranzate']


# <span style="color:red">Problemi di clustering</span>
## <span style="color:blue">Definizione:</span>
### Il clustering raggruppa insiemi di oggetti con cui bisogna relazionarsi, per un qualche futuro utilizzo o per trovare più facilmente gli oggetti al momento del bisogno.
### Un cluster è un insieme di oggetti che presentano tra loro delle similarità, ma che, per contro, presentano dissimilarità con oggetti di altri cluster.
### Ad ogni cluster è possibile associare un centroide, ossia un punto dello spazio euclideo posto al centro del cluster e rappresentativo di tutti gli elementi appartenenti allo stesso cluster.
## <span style="color:blue">Obiettivo:</span>
### Minimizzare la somma delle distanze di ogni nodo dal suo centroide.
## <span style="color:blue">Formulazione:</span>
### La formulazione di un problema di clustering è simile a quella di un problema di P-Mediana, dove come obiettivo, anzichè minimizzare il costo di afferenza, si vuole minimizzare la distanza dei nodi dai centroidi (mediane).

\begin{equation}
\text{Min} \quad Z = \sum_{(i \in I,j \in J)} d_{ij} \cdot x_{ij}
\end{equation}


# <span style="color:red">Rappresentazione dei cluster:</span>

### Generazione della matrice di assegnamento, dove viene associato ogni nodo al centroide più vicino, andando così a definire i cluster:

In [13]:
def assegnamento(distance_matrix, cities_names, centroids):

    cluster_matrix = {}
    for i in cities_names:
        if not(i in centroids):
            min_val = 100000000
            for j in centroids:
                if distance_matrix[(i, j)] < min_val:
                    min_val = distance_matrix[(i, j)]
                    centroid = j
            cluster_matrix[(i, centroid)] = distance_matrix[(i, centroid)]
    return cluster_matrix

cluster_matrix = assegnamento(distance_matrix, cities_names, medians)

### Rappresentazione dei cluster sulla mappa:

In [14]:
import random
import folium.vector_layers
import matplotlib.colors as colors

def map_generator(cluster_matrix, centroids, cities_coords):

    lat, lng = cities_coords[centroids[0]]
    colors_array = []
    for i in range(len(centroids)):
        colors_array.append([random.random(), random.random(), random.random()])
    map_clusters = folium.Map(location=[lat, lng], zoom_start=4)
    cont = 0
    for k in centroids:
        for i, j in cluster_matrix:
            if j == k:
                lat, lng = cities_coords[i]
                folium.vector_layers.CircleMarker(
                    [lat, lng],
                    radius=10,
                    tooltip='Nodo: ' + str(i) + ' Lat: ' + str(lat) + ' Lng: ' + str(lng),
                    color=colors.to_hex(colors_array[cont]),
                    fill=True,
                    fill_color=colors.to_hex(colors_array[cont]),
                    fill_opacity=0.9
                ).add_to(map_clusters)
        cont = cont + 1
        lat, lng = cities_coords[k]
        folium.vector_layers.Marker(
            [lat, lng],
            radius=10,
            tooltip='Centroide: ' + str(k) + ' Lat: ' + str(lat) + ' Lng: ' + str(lng)
        ).add_to(map_clusters)
    return map_clusters

map_cl = map_generator(cluster_matrix, medians, cities_coords)
map_cl

In [14]:
#FINE